## Problem Set 7
### MACS 30100, Dr. Evans
### Leoson Hoay

**Importing necessary packages:**

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

### Problem (1) 
**Classifier horse race.**

**(a)**

In [2]:
auto_df = pd.read_csv('../../../ProblemSets/PS7/Auto.csv', na_values='?')

In [3]:
auto_df.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      5
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

In [4]:
auto_df = auto_df.dropna()

In [5]:
mpg_median = auto_df['mpg'].median()
# create binary variable
auto_df['mpg_high'] = auto_df['mpg'].apply(lambda x: 1 if x >= mpg_median else 0)

In [6]:
auto_df.head(20)

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0
5,15.0,8,429.0,198.0,4341,10.0,70,1,ford galaxie 500,0
6,14.0,8,454.0,220.0,4354,9.0,70,1,chevrolet impala,0
7,14.0,8,440.0,215.0,4312,8.5,70,1,plymouth fury iii,0
8,14.0,8,455.0,225.0,4425,10.0,70,1,pontiac catalina,0
9,15.0,8,390.0,190.0,3850,8.5,70,1,amc ambassador dpl,0


In [7]:
auto_df['const'] = 1

In [8]:
X = auto_df[['const', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y = auto_df['mpg_high'].values

In [9]:
kf_log = KFold(n_splits=4, shuffle=True, random_state=171276)
kf_log.get_n_splits(X)
MSE_vec_kf = np.zeros(4)
true_MSE_kf = np.zeros(4)
false_MSE_kf = np.zeros(4)

k_ind = int(0)
for train_index, test_index in kf_log.split(X):
    print('Fold', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    
    true_vals = []
    true_preds = []
    false_vals = []
    false_preds = []
    for i, p in enumerate(y_pred):
        if p == 1:
            true_preds.append(p)
            true_vals.append(y_test[i])
        else:
            false_preds.append(p)
            false_vals.append(y_test[i])
    true_vals = np.asarray(true_vals)
    true_preds = np.asarray(true_preds)
    false_vals = np.asarray(false_vals)
    false_preds = np.asarray(false_preds)
    true_MSE_kf[k_ind] = ((true_vals - true_preds) ** 2).mean()
    false_MSE_kf[k_ind] = ((false_vals - false_preds) ** 2).mean()
     
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('MSE for mpg_high = 1 ->', true_MSE_kf[k_ind])
    print('MSE for mpg_high = 0 ->', false_MSE_kf[k_ind])
    print('')

    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('Model MSE (K-Folds) =', MSE_kf)
print('Model SD =', MSE_kf_std)
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()
print('Error rate for mpg_high => 1 -', true_MSE) 
print('Error rate for mpg_high => 0 -', false_MSE)

Fold 0
MSE for test set 0  is 0.07142857142857142
MSE for mpg_high = 1 -> 0.043478260869565216
MSE for mpg_high = 0 -> 0.09615384615384616

Fold 1
MSE for test set 1  is 0.17346938775510204
MSE for mpg_high = 1 -> 0.16666666666666666
MSE for mpg_high = 0 -> 0.18

Fold 2
MSE for test set 2  is 0.08163265306122448
MSE for mpg_high = 1 -> 0.07692307692307693
MSE for mpg_high = 0 -> 0.08695652173913043

Fold 3
MSE for test set 3  is 0.10204081632653061
MSE for mpg_high = 1 -> 0.16666666666666666
MSE for mpg_high = 0 -> 0.022727272727272728

Model MSE (K-Folds) = 0.10714285714285714
Model SD = 0.03984821263217681
Error rate for mpg_high => 1 - 0.11343366778149386
Error rate for mpg_high => 0 - 0.09645941015506232


**(b)**

In [10]:
auto_tree = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True, oob_score=True, random_state=25)
auto_tree.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [11]:
decision = auto_tree.oob_decision_function_[:,1]
binary = np.vectorize(lambda x: int(x >= .5))
ypred = binary(decision)

In [12]:
MSE_forest = mean_squared_error(y, ypred)
print('MSE of random forest model =', MSE_forest)

MSE of random forest model = 0.07908163265306123


In [13]:
tree_df = pd.DataFrame({'y':y,'ypred':ypred})
tree_df['MSE'] = tree_df.apply(lambda x: (x['y']-x['ypred'])**2, axis=1)
print('MSE for mpg_high = 1 ->', tree_df.groupby(y)['MSE'].mean()[1])
print('MSE for mpg_high = 0 ->', tree_df.groupby(y)['MSE'].mean()[0])

MSE for mpg_high = 1 -> 0.07142857142857142
MSE for mpg_high = 0 -> 0.08673469387755102


**(c)**

In [14]:
kf_svm = KFold(n_splits=4, shuffle=True, random_state=171276)
kf_svm.get_n_splits(4)
MSE_vec_kf = np.zeros(4)
true_MSE_kf = np.zeros(4)
false_MSE_kf = np.zeros(4)

k_ind = int(0)
for train_index, test_index in kf_log.split(X):
    print('Fold', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = svm.SVC(kernel='rbf', C=1, gamma=.2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    
    true_vals = []
    true_preds = []
    false_vals = []
    false_preds = []
    for i, p in enumerate(y_pred):
        if p == 1:
            true_preds.append(p)
            true_vals.append(y_test[i])
        else:
            false_preds.append(p)
            false_vals.append(y_test[i])
    
    if true_vals and true_preds:
        true_vals = np.asarray(true_vals)
        true_preds = np.asarray(true_preds)
        
        true_MSE_kf[k_ind] = ((true_vals - true_preds) ** 2).mean()
    else:
        true_MSE_kf[k_ind] = 0.0
    
    if false_vals and false_preds:
        false_preds = np.asarray(false_preds)
        false_vals = np.asarray(false_vals)
        
        false_MSE_kf[k_ind] = ((false_vals - false_preds) ** 2).mean()
    else:
        false_MSE_kf[k_ind] = 0.0
     
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('MSE for mpg_high = 1 ->', true_MSE_kf[k_ind])
    print('MSE for mpg_high = 0 ->', false_MSE_kf[k_ind])
    print('')

    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('Model Estimate (K-Folds) =', MSE_kf)
print('Model SD =', MSE_kf_std)
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()
print('Error rate for mpg_high = 1 ->', true_MSE)
print('Error rate for mpg_high = 0 ->', false_MSE)

Fold 0
MSE for test set 0  is 0.4897959183673469
MSE for mpg_high = 1 -> 0.0
MSE for mpg_high = 0 -> 0.4948453608247423

Fold 1
MSE for test set 1  is 0.45918367346938777
MSE for mpg_high = 1 -> 0.0
MSE for mpg_high = 0 -> 0.4787234042553192

Fold 2
MSE for test set 2  is 0.5
MSE for mpg_high = 1 -> 0.0
MSE for mpg_high = 0 -> 0.5157894736842106

Fold 3
MSE for test set 3  is 0.5306122448979592
MSE for mpg_high = 1 -> 0.5306122448979592
MSE for mpg_high = 0 -> 0.0

Model Estimate (K-Folds) = 0.4948979591836735
Model SD = 0.025510204081632664
Error rate for mpg_high = 1 -> 0.1326530612244898
Error rate for mpg_high = 0 -> 0.372339559691068


**(d)**

Using **Random Forests** seems to afford the highest level of accuracy, just by comparing model MSE values and factor MSE values as opposed to the Logistic Regression model, as well as the Support Vector Machines - which seems to perform much more poorly compared to the other two on accuracy. 

Since we are using disparate methodologies to obtain the model MSE, the performance of these models relative to one another are difficult to generalize. It may be wise to standardize the mean calculation in each specific method - all OOB/K-Folds - to elucidate a better comparison. It is reasonable for now, to hypothesize that Random Forests and Logistic Regression perform similarly well for this dataset.